First build rectangular distribution.

In [ ]:
# import the module build in a different folder
from scattering_structure.scattering_structure import ScatteringStructure

In [ ]:
rect = ScatteringStructure(geometry={'type': 'box', 'lx': 200, 'ly': 200},
                           arrangement={'type':'rectangular', 'dist': 6.44},
                           scatterer_radius=1.343
                           )
rect.plot_distribution()
rect_rms = rect.rms()
rect_density = rect.density()

# print for further use
print('Density = ', rect_density, '/um^2')

Then build a poisson disc distribution on top of that. Which mimiks the same density
#### Note:
The geometry is now given as circle i.e., the distribution will automatically be reduced to a circle inside the class.

In [ ]:
pois = ScatteringStructure(geometry={'type': 'circle', 'lx': 200, 'ly': 200,
                                     'circle_radius': 90},
                           arrangement={'type':'poisson_disc', 'optimization': True,
                                        'measure_of_merit':'density',
                                        'target_mom': rect_density,
                                        'optimization_outer_n': 4,
                                        'optimization_inner_n': 8},
                           scatterer_radius=1.343
                           )
pois.plot_distribution()
pois_density = pois.density()

# print for further use
print('Mean of RMS(dist) = ', pois_density, '/um^2')

Now plot the distribution after it's been reduced to the desired geometry.
#### Note:
The points have also been shifted to lie around the origin. Making later exporting to Lumerical or gds easier.

In [ ]:
pois.plot_device()

Save the distribution to a file

In [ ]:
pois.save_device(filepath='first_device.txt')

### Load first device and it's distribution

In [ ]:
load_pois =  ScatteringStructure(geometry={'type': 'load_from_file'},
                                 arrangement={'type': 'load_from_file', 
                                              'filepath': 'first_device.txt'},
                                 scatterer_radius=1.343
                           )

Let's try plotting the distribution again

In [ ]:
load_pois.plot_distribution()

## Create structure inside lumerical

In [1]:
import importlib.util
import os

# Add the DLL directory
os.add_dll_directory("C:\\Program Files\\Lumerical\\v232\\api\\python")

# Define the module name and file path
module_name = "lumapi"
file_path = "C:\\Program Files\\Lumerical\\v232\\api\\python\\lumapi.py"

def load_module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

# Load the module
lumapi = load_module_from_file(module_name, file_path)

C:\Program Files\Lumerical\v232\api\python\lumapi.py:882: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


In [2]:
## Constants
wavelength = 1.55 # central wavelength 
block_length = 210
block_width = 210


WG = {
    "length": block_length,
    "width": block_width,
    "height": 0.22,
    "matname": "Si3N4 (Silicon Nitride) - Luke"
}

BOX = {
    "length": block_length,
    "width": block_width,
    "height": 3,
    "matname": "SiO2 (Glass) - Palik"
}

SUB = {
    "length": block_length,
    "width": block_width,
    "height": 1,
    "matname": "Si (Silicon) - Palik"
}

AIR = {
    "length": block_length,
    "width": block_width,
    "height": 5,
    "matname": "etch"
}


#### SiO2, Si3N4, Air stack

In [3]:
fdtd = lumapi.FDTD()
# SiO2
fdtd.addrect()
fdtd.set('name', 'SUB')
coordinates = {"x": 0,
                   "x span": SUB["length"] * 1e-6,
                   "y": 0,
                   "y span": SUB["width"] * 1e-6,
                   "z min": 0,
                   "z max": SUB["height"] * 1e-6}
fdtd.set(coordinates)
fdtd.select('SUB')
fdtd.set('material', SUB["matname"])
# default settings
fdtd.set('color opacity', 0.5)
fdtd.set('override mesh order from material database', 1)
fdtd.set('mesh order', 3)
# Si3N4
fdtd.addrect()
fdtd.set('name', 'WG')
coordinates = {"x": 0,
                   "x span": WG["length"] * 1e-6,
                   "y": 0,
                   "y span": WG["width"] * 1e-6,
                   "z min": WG["height"] * 1e-6,
                   "z max": (SUB["height"] + WG["height"]) * 1e-6}
fdtd.set(coordinates)
fdtd.select('WG')
fdtd.set('material', WG["matname"])
# default settings
fdtd.set('color opacity', 0.5)
fdtd.set('override mesh order from material database', 1)
fdtd.set('mesh order', 3)
# Air
fdtd.addrect()
fdtd.set('name', 'AIR')
coordinates = {"x": 0,
                   "x span": AIR["length"] * 1e-6,
                   "y": 0,
                   "y span": AIR["width"] * 1e-6,
                   "z min": AIR["height"] * 1e-6,
                   "z max": (SUB["height"] + WG["height"] + AIR["height"]) * 1e-6}
fdtd.set(coordinates)
fdtd.select('AIR')
fdtd.set('material', AIR["matname"])
# default settings
fdtd.set('color opacity', 0.5)
fdtd.set('override mesh order from material database', 1)
fdtd.set('mesh order', 3)